In [3]:
import pandas as pd
from pandasql import sqldf
big_file = 'pai_task_table.csv'
small_file = 'small_csv.csv'

columns = [
    'job_name', 'task_name', 'inst_num', 'status',
    'start_time', 'end_time', 'plan_cpu', 'plan_mem', 'plan_gpu', 'gpu_type'
]
pysqldf = lambda q: sqldf(q, globals())

In [4]:
task_df = pd.read_csv(f'../../{big_file}', names=columns)

### Find all unique task names

In [22]:
query = '''
select task_name, count(*) as cnt
from task_df
group by task_name
order by cnt desc
'''
pysqldf(query)

,task_name,cnt
0,tensorflow,621415
1,worker,275785
2,ps,183283
3,PyTorchWorker,110784
4,xComputeWorker,27402
5,evaluator,17210
6,TensorboardTask,10681
7,ReduceTask,4136
8,DecoderWorker,4136
9,JupyterTask,2066


### Find all jobs that has an evaluator task, and see how many tasks it has
Conclusion: 2 ~ 4 tasks

In [31]:
# get all jobs containing the evaluator task, and see how many tasks it has
query = '''
select job_name, count(*) cnt
from task_df
where job_name in (
select distinct job_name from task_df
where task_name = 'evaluator'
)
group by job_name
order by cnt desc
'''

pysqldf(query)

,job_name,cnt
0,ff8ee30fa85e41e500a19fc6,4
1,f9e972a35ef592e35bf7979a,4
2,f9c6d5363510ad32568efa90,4
3,f6be056fdfe3dab9e1706a10,4
4,f3751b816c8b78113b6a4429,4
...,...,...
17205,0666c7a24504f95a5665a6a9,2
17206,058c4bcefa43a8b22ff3678a,2
17207,049f31106b289d2c74de973d,2
17208,03d55b51b2abcc5ab0ca08c3,2


### For an evaluator task, how many instances does it spawn?
Conclusion: each evaluator task spawns eaxactly 1 instance

In [24]:
# for an evaluator task, how many instances does it spawn and how much gpu per instance does it acquire?
# conclusion: each evaluator task only launches one instance

query = '''
select DATETIME(start_time, 'unixepoch') as start_time, DATETIME(end_time, 'unixepoch') as end_time, inst_num
from task_df
where task_name = 'evaluator' and status != 'Waiting'
order by inst_num asc
'''
pysqldf(query)

,start_time,end_time,inst_num
0,1970-02-10 10:12:05,1970-02-10 10:35:43,1.0
1,1970-02-03 16:17:36,None,1.0
2,1970-01-14 19:02:44,1970-01-15 03:27:31,1.0
3,1970-01-29 22:30:56,1970-01-29 23:58:22,1.0
4,1970-01-19 14:00:11,1970-01-19 14:42:02,1.0
...,...,...,...
17193,1970-02-26 02:06:27,1970-02-26 02:14:51,1.0
17194,1970-02-26 04:10:46,1970-02-26 04:39:14,1.0
17195,1970-03-11 11:25:42,1970-03-11 13:40:43,1.0
17196,1970-02-25 03:18:07,1970-02-25 03:19:41,1.0


### For an evaluator task, how much gpu does it acquire?
Conclusion: mostly 25%, but there are also tasks demanding up to 400% gpu

In [27]:
# for an evaluator task, how much gpu does it acquire?
# conclusion: mostly 25%, but there are also tasks demanding up to 400% gpu
query = '''
select plan_gpu, count(*) as cnt
from task_df
where task_name = 'evaluator'
group by plan_gpu
order by cnt desc
'''
pysqldf(query)

,plan_gpu,cnt
0,25.0,13020
1,NaN,3367
2,100.0,302
3,50.0,255
4,400.0,149
5,200.0,78
6,5.0,21
7,10.0,18


### For a job that has an evaluator task, how many evaluator tasks does it have?
Conclusion: each job has at most one evaluator task

In [32]:
# for a job that has an evaluator task, how many evaluator tasks does it have?
# conclusion: each job only has one evaluator task at most
query = '''
select job_name, count(*) as cnt
from task_df
where task_name = 'evaluator'
group by job_name
order by cnt desc
'''
pysqldf(query)

,job_name,cnt
0,fffa8826d76f50b27199a6a2,1
1,ffecfa2bcf1f5c4d5f92a17f,1
2,ffe66c9b305052cca0a8fc44,1
3,ffe62cdf82562b3f2afa8e45,1
4,ffdf47e88a9531caa321b06f,1
...,...,...
17205,0013befc8e8a32be8f447cac,1
17206,00101bac8991750d3adc7ac9,1
17207,00066be6b189aa0ae9f09e02,1
17208,0004066bf4f23fa011a0caee,1


In [29]:
job_df_columns = ['job_name', 'inst_id', 'user', 'status', 'start_time', 'end_time']
job_df = pd.read_csv('../../pai_job_table.csv', names=job_df_columns)


In [ ]:
# Get all the jobs which have an evaluator task
query = '''
select *
from job_df
where exists (
    select *
    from task_df
    where task_name = 'evaluator' and task_df.job_name = 
)
'''